In [2]:
import rby1_sdk as rb
import argparse
import numpy as np

In [3]:

np.set_printoptions(precision=3, suppress=True, floatmode="fixed")


def get_joints(address = "192.168.30.1:50051", model = "a", power = ".*"):
    robot = rb.create_robot(address, model)
    robot.connect()
    if not robot.is_connected():
        print("Robot is not connected")
        exit(1)
    if not robot.is_power_on(power):
        rv = robot.power_on(power)
        if not rv:
            print("Failed to power on")
            exit(1)
    robot.reset_fault_control_manager()
    robot.enable_control_manager()

    robot_state = robot.get_state()
    body = [round(x, 3) for x in robot_state.position[2:8]]
    right_arm = [round(x, 3) for x in robot_state.position[8:15]]
    left_arm = [round(x, 3) for x in robot_state.position[15:22]]

    result = [
        body,
        right_arm,
        left_arm
    ]
    return result

In [4]:
_, _, left_arm = get_joints()
left_arm

[np.float64(-0.844),
 np.float64(0.329),
 np.float64(-0.334),
 np.float64(-1.617),
 np.float64(0.055),
 np.float64(1.625),
 np.float64(0.0)]

In [16]:
import rby1_sdk as rb
ADDRESS = "192.168.30.1:50051"
MODEL = "a"

robot = rb.create_robot(ADDRESS, MODEL)
model = robot.model()
robot.connect()
dyn_model = robot.get_dynamics()
dyn_state = dyn_model.make_state(["base", "link_torso_5", "ee_right", "ee_left"], model.robot_joint_names)
BASE_INDEX, LINK_TORSO_5_INDEX, EE_RIGHT_INDEX, EE_LEFT_INDEX = 0, 1, 2, 3
model.robot_joint_names


['right_wheel',
 'left_wheel',
 'torso_0',
 'torso_1',
 'torso_2',
 'torso_3',
 'torso_4',
 'torso_5',
 'right_arm_0',
 'right_arm_1',
 'right_arm_2',
 'right_arm_3',
 'right_arm_4',
 'right_arm_5',
 'right_arm_6',
 'left_arm_0',
 'left_arm_1',
 'left_arm_2',
 'left_arm_3',
 'left_arm_4',
 'left_arm_5',
 'left_arm_6',
 'head_0',
 'head_1']

In [17]:
dyn_state.set_q([0] * model.robot_dof)
dyn_model.compute_forward_kinematics(dyn_state)
T_base2left = dyn_model.compute_transformation(dyn_state, BASE_INDEX, EE_LEFT_INDEX)
print(T_base2left)

[[1.     0.     0.     0.    ]
 [0.     1.     0.     0.22  ]
 [0.     0.     1.     0.6832]
 [0.     0.     0.     1.    ]]


In [22]:
dyn_state.set_q(robot.get_state().position)
dyn_model.compute_forward_kinematics(dyn_state)
T_base2left = dyn_model.compute_transformation(dyn_state, BASE_INDEX, EE_LEFT_INDEX)
R = T_base2left[:3,:3]
p = T_base2left[:3,3]
print("T_base2left: \n", T_base2left, end="\n\n")
print("R: \n", R, end="\n\n")
print("p: \n", p, end="\n\n")


T_base2left: 
 [[ 0.64995693  0.30627149 -0.69552409  0.53229961]
 [-0.58614079  0.78455624 -0.20226339  0.11352804]
 [ 0.48373026  0.53913753  0.68944598  1.11000654]
 [ 0.          0.          0.          1.        ]]

R: 
 [[ 0.64995693  0.30627149 -0.69552409]
 [-0.58614079  0.78455624 -0.20226339]
 [ 0.48373026  0.53913753  0.68944598]]

p: 
 [0.53229961 0.11352804 1.11000654]

